In [2]:
!python -m pip install pandas
!python -m pip install numpy
!python -m pip install matplotlib
!python -m pip install seaborn


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
!python -m pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
    --------------------------------------- 0.3/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 2.0 MB/s eta 0:00:06
   --- ------------------------------------ 1.0/11.1 MB 2.0 MB/s eta 0:00:06
   ---- ----------------------------------- 1.3/11.1 MB 1.9 MB/s eta 0:00:06
   ------ --------------------------------- 1.8/11.1 MB 2.0 MB/s eta 0:00:05
   ------- -------------------------------- 2.1/11.1 MB 1.9 MB/s eta 0:00:05
   ------- -------------------------------- 2.1/11.1 MB 1.9 MB/s eta 0:00:05
   ------- -------------------------------- 2.1/11.1 MB 1.9 MB/s eta 0:00:05
   -------- ------------------------------- 2.4/11.1 MB 1.3 MB/s eta 0:00:07
   ---------- ----------------------------- 2.9/11.1 MB 1.4 MB/s eta 0:00:06
   ----------- ---------------------------- 3.1/11.1 MB 1.5 MB/s eta 0:00:06
   ---------

In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


customers_df = pd.read_csv('Customers.csv')
products_df = pd.read_csv('Products.csv')
transactions_df = pd.read_csv('Transactions.csv')


customer_transactions = transactions_df.merge(customers_df, on='CustomerID')


customer_features = customer_transactions.groupby('CustomerID').agg(
    total_spend=('TotalValue', 'sum'),
    transaction_count=('TransactionID', 'count'),
    avg_transaction_value=('TotalValue', 'mean'),
    unique_products_bought=('ProductID', lambda x: len(set(x)))
).reset_index()


customer_features = customer_features.merge(customers_df[['CustomerID', 'Region']], on='CustomerID')


customer_features = pd.get_dummies(customer_features, columns=['Region'], drop_first=True)


scaler = StandardScaler()
customer_features[['total_spend', 'transaction_count', 'avg_transaction_value', 'unique_products_bought']] = scaler.fit_transform(
    customer_features[['total_spend', 'transaction_count', 'avg_transaction_value', 'unique_products_bought']]
)

customer_features.head()


,CustomerID,total_spend,transaction_count,avg_transaction_value,unique_products_bought,Region_Europe,Region_North America,Region_South America
0,C0001,-0.061701,-0.011458,-0.070263,0.050047,False,False,True
1,C0002,-0.877744,-0.467494,-0.934933,-0.424204,False,False,False
2,C0003,-0.405857,-0.467494,-0.026271,-0.424204,False,False,True
3,C0004,1.032547,1.356650,-0.076769,1.472798,False,False,True
4,C0005,-0.783929,-0.923530,-0.040028,-0.898455,False,False,False


In [7]:

features = customer_features.drop(columns=['CustomerID'])


cosine_sim = cosine_similarity(features)


cosine_sim_df = pd.DataFrame(cosine_sim, index=customer_features['CustomerID'], columns=customer_features['CustomerID'])

cosine_sim_df.head()


CustomerID,C0001,C0002,C0003,C0004,C0005,C0006,C0007,C0008,C0009,C0010,...,C0191,C0192,C0193,C0194,C0195,C0196,C0197,C0198,C0199,C0200
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,1.000000,0.072333,0.803886,0.403149,0.011072,0.422552,-0.035423,0.039413,0.061549,0.063597,...,0.945781,0.681421,-0.026602,0.036812,0.797773,-0.088659,0.016591,0.035498,0.055110,-0.089810
C0002,0.072333,1.000000,0.435975,-0.593923,0.713370,-0.497651,0.251699,-0.276681,0.925340,0.843785,...,0.354265,0.769439,0.347354,-0.198695,-0.359681,-0.566947,0.655151,0.842915,0.794141,-0.889077
C0003,0.803886,0.435975,1.000000,-0.219208,0.600015,0.475748,0.508976,-0.488632,0.421638,0.348681,...,0.799561,0.875107,0.538647,-0.412099,0.283743,0.056097,0.502505,0.526942,0.363689,-0.201125
C0004,0.403149,-0.593923,-0.219208,1.000000,-0.909109,0.010342,-0.816165,0.795410,-0.584862,-0.469123,...,0.303345,-0.254898,-0.852451,0.673851,0.873338,-0.181055,-0.754021,-0.773607,-0.500801,0.215813
C0005,0.011072,0.713370,0.600015,-0.909109,1.000000,0.140830,0.856529,-0.827540,0.692806,0.569408,...,0.109681,0.604647,0.904113,-0.698277,-0.593475,0.114252,0.836139,0.874189,0.596118,-0.317883


In [11]:

lookalikes = {}


for customer_id in customer_features['CustomerID'].iloc[:20]:
    
    similar_customers = cosine_sim_df[customer_id].sort_values(ascending=False)[1:4]  # Exclude the customer itself
    
  
    results = list(zip(similar_customers.index, similar_customers.values))
    while len(results) < 3:
        results.append((None, 0))  
    
 
    lookalikes[customer_id] = [item for sublist in results for item in sublist]


lookalike_df = pd.DataFrame.from_dict(
    lookalikes, 
    orient='index', 
    columns=['Lookalike 1', 'Score 1', 'Lookalike 2', 'Score 2', 'Lookalike 3', 'Score 3']
)


lookalike_df.reset_index(inplace=True)
lookalike_df.rename(columns={'index': 'CustomerID'}, inplace=True)


lookalike_df.to_csv('Lookalike.csv', index=False)

lookalike_df.head()


,CustomerID,Lookalike 1,Score 1,Lookalike 2,Score 2,Lookalike 3,Score 3
0,C0001,C0137,0.999762,C0152,0.999512,C0107,0.964257
1,C0002,C0043,0.987504,C0142,0.977492,C0097,0.961819
2,C0003,C0133,0.988559,C0052,0.943816,C0112,0.938337
3,C0004,C0108,0.986197,C0113,0.973500,C0155,0.960225
4,C0005,C0159,0.999295,C0123,0.998508,C0178,0.998475
